In [245]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd
import glob
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [247]:
# os.chdir("ECB Press Conference")

In [248]:
file_list = os.listdir()
file_list = [file for file in file_list if file.endswith(".pdf")] 
file_list.sort()
print(file_list)
file_dict = {}
for i in range(len(file_list)):
    file_dict[file_list[i]] = i

['april 18.pdf', 'april 19.pdf', 'april 20.pdf', 'april 21.pdf', 'april 22.pdf', 'december 18.pdf', 'december 19.pdf', 'december 20.pdf', 'december 21.pdf', 'december 22.pdf', 'february 22.pdf', 'february 23.pdf', 'january 18.pdf', 'january 19.pdf', 'january 20.pdf', 'january 21.pdf', 'july 18.pdf', 'july 19.pdf', 'july 20.pdf', 'july 21.pdf', 'july 22.pdf', 'july 23.pdf', 'june 18.pdf', 'june 19.pdf', 'june 20.pdf', 'june 21.pdf', 'june 22.pdf', 'june 23.pdf', 'march 18.pdf', 'march 19.pdf', 'march 20.pdf', 'march 21.pdf', 'march 22.pdf', 'march 23.pdf', 'may 23.pdf', 'october 18.pdf', 'october 19.pdf', 'october 20.pdf', 'october 21.pdf', 'october 22.pdf', 'october 23.pdf', 'september 18.pdf', 'september 19.pdf', 'september 20.pdf', 'september 21.pdf', 'september 22.pdf', 'september 23.pdf']


In [249]:
print(file_list)

['april 18.pdf', 'april 19.pdf', 'april 20.pdf', 'april 21.pdf', 'april 22.pdf', 'december 18.pdf', 'december 19.pdf', 'december 20.pdf', 'december 21.pdf', 'december 22.pdf', 'february 22.pdf', 'february 23.pdf', 'january 18.pdf', 'january 19.pdf', 'january 20.pdf', 'january 21.pdf', 'july 18.pdf', 'july 19.pdf', 'july 20.pdf', 'july 21.pdf', 'july 22.pdf', 'july 23.pdf', 'june 18.pdf', 'june 19.pdf', 'june 20.pdf', 'june 21.pdf', 'june 22.pdf', 'june 23.pdf', 'march 18.pdf', 'march 19.pdf', 'march 20.pdf', 'march 21.pdf', 'march 22.pdf', 'march 23.pdf', 'may 23.pdf', 'october 18.pdf', 'october 19.pdf', 'october 20.pdf', 'october 21.pdf', 'october 22.pdf', 'october 23.pdf', 'september 18.pdf', 'september 19.pdf', 'september 20.pdf', 'september 21.pdf', 'september 22.pdf', 'september 23.pdf']


In [4]:
import PyPDF2

def remove_pages(input_pdf, output_pdf, pages_to_remove):
    """
    This function removes specific pages from a PDF and saves the result to a new PDF.

    Args:
        input_pdf: Path to the input PDF file.
        output_pdf: Path to the output PDF file.
        pages_to_remove: A list of page numbers to remove (1-based indexing).
    """
    reader = PyPDF2.PdfReader(input_pdf)
    writer = PyPDF2.PdfWriter()
    # remove pages after pagest_to_remove
    for i, page in enumerate(reader.pages):
        if i + 1 <= pages_to_remove:
            writer.add_page(page)
    with open(output_pdf, "wb") as f:
        writer.write(f)
    # delete the original file
    os.remove(input_pdf)

# run remove_pages for each file in file_list
for file in file_list:
    input_pdf = file
    output_pdf = file[:-4] + "_.pdf"
    pages_to_remove = 4

    remove_pages(input_pdf, output_pdf, pages_to_remove)
    print(f"Successfully removed pages {pages_to_remove} from '{input_pdf}' and saved the result to '{output_pdf}'")

Successfully removed pages 4 from 'april 18.pdf' and saved the result to 'april 18_.pdf'
Successfully removed pages 4 from 'april 19.pdf' and saved the result to 'april 19_.pdf'
Successfully removed pages 4 from 'april 20.pdf' and saved the result to 'april 20_.pdf'
Successfully removed pages 4 from 'april 21.pdf' and saved the result to 'april 21_.pdf'
Successfully removed pages 4 from 'april 22.pdf' and saved the result to 'april 22_.pdf'
Successfully removed pages 4 from 'december 18.pdf' and saved the result to 'december 18_.pdf'
Successfully removed pages 4 from 'december 19.pdf' and saved the result to 'december 19_.pdf'
Successfully removed pages 4 from 'december 20.pdf' and saved the result to 'december 20_.pdf'
Successfully removed pages 4 from 'december 21.pdf' and saved the result to 'december 21_.pdf'
Successfully removed pages 4 from 'december 22.pdf' and saved the result to 'december 22_.pdf'
Successfully removed pages 4 from 'february 22.pdf' and saved the result to 'feb

In [19]:
# for filename in os.listdir():
#     new_filename = filename.replace("_", "")
#     os.rename(filename, new_filename)

In [250]:
text_dictionary = {}

def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

# opening each file in each folder
def make_text_dictionary():
    folder_names = file_list
    # show this as tqdm
    for i in folder_names:
        print(i)
        text = get_text(i)
        text_dictionary[file_dict[i]] = text
    os.chdir('/home/uttkarsh/IP/Supervised/ECB Press Conference')
    # dump the text_dictionary into a json file
    with open('text_dictionary.json', 'w') as fp:
        json.dump(text_dictionary, fp)
    return None

make_text_dictionary()

april 18.pdf
april 19.pdf
april 20.pdf
april 21.pdf
april 22.pdf
december 18.pdf
december 19.pdf
december 20.pdf
december 21.pdf
december 22.pdf
february 22.pdf
february 23.pdf
january 18.pdf
january 19.pdf
january 20.pdf
january 21.pdf
july 18.pdf
july 19.pdf
july 20.pdf
july 21.pdf
july 22.pdf
july 23.pdf
june 18.pdf
june 19.pdf
june 20.pdf
june 21.pdf
june 22.pdf
june 23.pdf
march 18.pdf
march 19.pdf
march 20.pdf
march 21.pdf
march 22.pdf
march 23.pdf
may 23.pdf
october 18.pdf
october 19.pdf
october 20.pdf
october 21.pdf
october 22.pdf
october 23.pdf
september 18.pdf
september 19.pdf
september 20.pdf
september 21.pdf
september 22.pdf
september 23.pdf


In [251]:
os.chdir('/home/uttkarsh/IP/Supervised/ECB Press Conference')
with open('text_dictionary.json', 'r') as fp:
    corpus = json.load(fp)
corpus = list(corpus.values())

#convert corpus to df
corpus = pd.DataFrame(corpus, columns = ['text'])
corpus.head()
corpus.shape

(47, 1)

In [252]:
# remove websites
corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'http\S+', '', x))

# clean corpus: remove punctuation 
corpus['text'] = corpus['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# remove single lettered words
corpus = corpus[corpus['text'].apply(lambda x: len(x) > 1)]

# remove numbers
corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'\d+', '', x))

# remove 2 letter words
corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'\b\w{1,2}\b', '', x))

# join all 'per' 'cent' words coming together
corpus['text'] = corpus['text'].apply(lambda x: re.sub(r'per cent', 'percent', x))


In [253]:
print(corpus.head())

                                                text
0     PRESS CONFERENCE PRESS CONFERENCEMario Drag...
1     PRESS CONFERENCE INTRODUCT ORY STATEMENTPRE...
2     PRESS CONFERENCE INTRODUCT ORY STATEMENTPRE...
3     PRESS CONFERENCE INTRODUCT ORY STATEMENTPRE...
4     PRESS CONFERENCE MONET ARY POLICY  STATEMEN...


In [289]:
class Lemmatizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()

  def __call__(self, text):
    return [self.wnl.lemmatize(word) for word in word_tokenize(text)]
  
stops = set(stopwords.words("english"))
vectorizer = TfidfVectorizer(tokenizer=Lemmatizer(), stop_words=list(stops), max_features=40000)
data = vectorizer.fit_transform(corpus['text'])

/home/uttkarsh/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/uttkarsh/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(


In [290]:
tf_idf_df = pd.DataFrame(data.toarray(), columns=vectorizer.get_feature_names_out())

In [292]:
print(tf_idf_df)

       abate  abatedsomewhat   abating      able    abouta  abouthalf  \
0   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
1   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
2   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
3   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
4   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
5   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
6   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
7   0.000000        0.000000  0.038842  0.000000  0.000000   0.000000   
8   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
9   0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
10  0.000000        0.040201  0.000000  0.000000  0.000000   0.000000   
11  0.000000        0.000000  0.000000  0.000000  0.000000   0.000000   
12  0.000000        0.000000  0.000000  0.000000  0

In [256]:
tf_idf_df.to_csv('tf_idf_df.csv', index=False)

# import the csv file
tf_idf_df = pd.read_csv('tf_idf_df.csv', header=None)
# remove the first row
tf_idf_df = tf_idf_df.iloc[1:, :]

In [257]:
tf_idf_df.head()

,0,1,2,3,4,5,6,7,8,9,...,7331,7332,7333,7334,7335,7336,7337,7338,7339,7340
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0193662427547715,0.0,0.0,0.0,0.014939906268631934,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.03842978708702706,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.04795439370272785,0.0,0.0,0.0,0.05919034734076631,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.009957745606841026,0.020466556508895138,0.02109596352085013,0.0,0.01536361883891315,0.026077751510680994
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039148555561511963,...,0.0,0.0,0.0,0.0,0.03826926865413186,0.0786563726759387,0.08107528826028061,0.0,0.0,0.0


In [258]:
tf_idf_df

,0,1,2,3,4,5,6,7,8,9,...,7331,7332,7333,7334,7335,7336,7337,7338,7339,7340
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0193662427547715,0.0,0.0,0.0,0.014939906268631934,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.03842978708702706,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.04795439370272785,0.0,0.0,0.0,0.05919034734076631,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.009957745606841026,0.020466556508895138,0.02109596352085013,0.0,0.01536361883891315,0.026077751510680994
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039148555561511963,...,0.0,0.0,0.0,0.0,0.03826926865413186,0.0786563726759387,0.08107528826028061,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.04241200370119784,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.05182427360732776,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.02724508368114995,0.0,0.0,0.0,0.014011976118500547,0.0
8,0.0,0.0,0.03884153305406855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.037969141953371324,0.0,0.0,0.0,0.043936407469434384,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.009546447699252383,0.019621199316613564,0.020224609100153636,0.0,0.04418710504599695,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07996695678414817,0.01826214603280795,0.018823760917138867,0.0,0.013708835226223291,0.0


In [259]:
os.chdir('/home/uttkarsh/IP/Supervised')
# Assuming labels_df is your DataFrame with labels
# Make sure that the rows in labels_df align with the rows in tfidf_df
# get the Classification.xlxs
labels_df = pd.read_excel('Classification.xlsx')
labels_df.head()

,filename,deviation,Inflation,Interest Rates
0,September 21,1,Increase,Decrease
1,October 21,1,Increase,Decrease
2,December 21,1,Increase,Unchanged
3,April 22,1,Increase,Unchanged
4,June 22,1,Increase,Unchanged


In [260]:
tf_idf_df.shape

(47, 7341)

In [274]:
for col in tf_idf_df.select_dtypes(include=['object']).columns:
    tf_idf_df[col] = tf_idf_df[col].astype('category')

In [275]:
tf_idf_df = pd.get_dummies(tf_idf_df)

In [276]:
# sort the date by filename
labels_df = labels_df.sort_values(by=['filename']).reset_index(drop=True)

labels = labels_df['deviation']

labels

0     1
1     0
2     0
3     1
4     1
5     0
6     1
7     0
8     1
9     0
10    1
11    0
12    0
13    0
14    1
15    0
16    1
17    1
18    1
19    1
20    0
21    0
22    1
23    0
24    0
25    1
26    1
27    0
28    0
29    1
30    0
31    1
32    1
33    0
34    0
35    1
36    0
37    0
38    1
39    0
40    0
41    0
42    0
43    0
44    1
45    0
46    0
Name: deviation, dtype: int64

In [277]:
labels.shape

(47,)

In [288]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_df, labels, test_size=0.2, random_state=10000)

# Initializing the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

# Training the model
xgb_model.fit(X_train, y_train)

# Making predictions
predictions = xgb_model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.8
